In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

### Question 1: Use yfinance to Extract Stock Data

In [ ]:
tesla = yf.Ticker("TSLA")

In [ ]:
tesla_share_price_data = tesla.history(period="max")

In [ ]:
tesla_share_price_data.reset_index(inplace=True)
tesla_share_price_data.head()

### Question 2: Use Webscraping to Extract Tesla Revenue Data

In [ ]:
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

In [ ]:
soup = BeautifulSoup(html_data)

In [ ]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):
    if ('Tesla Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text

                tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index = True)


In [ ]:
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

In [ ]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

### Question 3: Use yfinance to Extract Stock Data

In [ ]:
gme = yf.Ticker('GME')

In [ ]:
gme_data = gme.history(period='max')

In [ ]:
gme_data.reset_index(inplace=True)
gme_data.head(5)

### Question 4: Use Webscraping to Extract GME Revenue Data

In [ ]:
url_2 = ' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'

In [ ]:
html_data_2 = requests.get(url_2).text

In [ ]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):
    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

    for row in rows:
        col = row.find_all('td')

        if col != []:
            date = col[0].text
            revenue = col[1].text.replace(',','').replace('$','')

            gme_revenue = pd.concat([gme_revenue,pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

In [ ]:
gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [ ]:
gme_revenue.tail()

### Question 5: Plot Tesla Stock Graph

In [ ]:
make_graph(tesla_share_price_data[['Date','Close']], tesla_revenue, 'Tesla')

### Question 6: Plot GameStop Stock Graph

In [ ]:
make_graph(gme_data[['Date','Close']], gme_revenue, 'GameStop')